# Этап 1: Градиентный бустинг

In [1]:
import pandas as pd

import time
import datetime

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
train_data = pd.read_csv('dataset/features.csv', index_col='match_id')
train_data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
# Удаляем признаки об исходе игры из train_data, так как они отсутствуют в тестовой выборке
# признаки для удаления duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire

drop_data = train_data.iloc[:, -6:] 
Y_train = train_data['radiant_win']
X_train = train_data.drop(drop_data, axis=1)

In [4]:
# Проверяем выборку на наличие пропусков
counts = X_train.count()
counts_na = counts[counts < len(X_train)]
print(counts_na)
print('='*40)
print(1 - counts_na[0:4]/len(X_train))

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64
first_blood_time       0.201100
first_blood_team       0.201100
first_blood_player1    0.201100
first_blood_player2    0.452402
dtype: float64


# Ответ 1:
Признаки first_blood_time, first_blood_team в 20 % случаев принимают пустые значения, поскольку событие "первая кровь" не успело произойти за первые 5 минут, количество пустых значений признака first_blood_player1 (игрок, причастный к событию) совпадает с first_blood_time, first_blood_team и равно 20 %, в то время как для признака first_blood_player2 (второй игрок, причастный к событию) количество пропущенных значений составляет 45 % выборки.
            

In [5]:
# Заменим пропуски на нули с помощью функции fillna()
X_train = X_train.fillna(0)


# Ответ 2:

Целевой переменной является признак radiant_win.

In [6]:
# градиентный бустинг
kf = KFold(n_splits=5, random_state=42, shuffle=True)
tuned_parameters = [{'n_estimators': [10, 20, 30, 50, 100]}]
clf = GradientBoostingClassifier()
GridCV = GridSearchCV(clf, param_grid = tuned_parameters, cv=kf, scoring='roc_auc')

start_time = datetime.datetime.now()
GridCV.fit(X_train, Y_train)
print('Time elapsed:', datetime.datetime.now() - start_time)

print("Mean train scores for 10, 20, 30, 50, 100 estimators:", GridCV.cv_results_['mean_train_score'])
print("Mean test scores for 10, 20, 30, 50, 100 estimators:", GridCV.cv_results_['mean_test_score'])
print("Mean fit time for 10, 20, 30, 50, 100 estimators:", GridCV.cv_results_['mean_fit_time'])


Time elapsed: 0:08:52.578965
Mean train scores for 10, 20, 30, 50, 100 estimators: [ 0.67374299  0.69341142  0.70201888  0.71197297  0.72627958]
Mean test scores for 10, 20, 30, 50, 100 estimators: [ 0.66485069  0.68246188  0.69000647  0.69749436  0.70622298]
Mean fit time for 10, 20, 30, 50, 100 estimators: [  4.94420891   9.11841593  13.62762842  21.80443826  43.81407871]


# Ответ 3:
 
 Среднее время кросс-валидации по 5 разбиениям для градиентного бустинга с 30 деревьями составило 13.44 секунд. Качество (метрика roc-auc), усредненное по 5 разбиениям, на тестовой выборке составило 0.69 для 30 деревьев.

# Ответ 4:

Увеличение количества деревьев в данном алгоритме позволяет улучшить качество модели, но незначительно. При использовании большого количества деревьев > 250, можно разбить признаки на части, и обучаться по частям, распараллелить алгоритм и обучать эти части одновременно, потом склеить части для вычисления метрики. Можно также обучать деревья параллельно.


# Этап 2: Логистическая регрессия

In [7]:
import pandas as pd
import numpy as np

import time
import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [18]:
train_data = pd.read_csv('dataset/features.csv', index_col='match_id')

# Удаляем признаки об исходе игры из train_data, так как они отсутствуют в тестовой выборке
# признаки для удаления duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
drop_data = train_data.iloc[:, -6:] 
Y_train = train_data['radiant_win']
train_data = train_data.drop(drop_data, axis=1)
X_train = train_data

# Заменим пропуски на нули с помощью функции fillna()
X_train = X_train.fillna(0)

# масштабирование признаков обучающей выборки
stdSc = StandardScaler()
X_train = stdSc.fit_transform(X_train)

In [9]:
def logistic_regressor_tuning(X, Y):
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
    tuned_parameters = [{'C': C}]
    clf = LogisticRegression(penalty='l2', max_iter=100)
    GridCV = GridSearchCV(clf, param_grid = tuned_parameters, cv=kf, scoring='roc_auc')

    start_time = datetime.datetime.now()
    GridCV.fit(X, Y)
    print('Time elapsed:', datetime.datetime.now() - start_time)
    print('='*40)
    print("Mean train scores for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]:", GridCV.cv_results_['mean_train_score'])
    print("Mean test scores for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]:", GridCV.cv_results_['mean_test_score'])
    print("Mean fit time for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]:", GridCV.cv_results_['mean_fit_time'])
    print('='*40)
    index_max_score = np.argmax(GridCV.cv_results_['mean_test_score'])
    print("Optimal C and score on test data:", C[index_max_score], GridCV.cv_results_['mean_test_score'][[index_max_score]])

# логистическая регрессия с L2 регуляризацией над всеми исходными признаками
logistic_regressor_tuning(X_train, Y_train)

Time elapsed: 0:01:26.682600
Mean train scores for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]: [ 0.71241966  0.71777456  0.71813887  0.71814973  0.71815039  0.71815044
  0.71815044]
Mean test scores for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]: [ 0.71135775  0.71636354  0.71655027  0.71652715  0.7165226   0.71652229
  0.7165223 ]
Mean fit time for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]: [ 1.62360353  1.64520245  2.24745889  2.35804315  2.4102448   2.49666529
  2.51548219]
Optimal C and score on test data: 0.01 [ 0.71655027]


# Ответ 1:
Качество логистической регрессии над всеми исходными признаками при С=0.01 составило 0.716550269726.
Качество (метрика auc_roc) градиентного бустинга при использовании разного количества деревьев (10, 20, 30, 50, 100) незначительно уступает логистической регрессии, отличия лежат в диапазоне от 1 до 6 %. Логистическая регрессия более склонна к underfitting и overfitting, возможно поэтому дает более оптимистичные прогнозы по сравнению с градиентным бустингом, но работает быстрее при том же качестве.

In [21]:
# удалим категориальные признаки
drop_data = train_data.loc[:, ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
                       'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']] 
X_train = train_data.drop(drop_data, axis=1)

# Заменим пропуски на нули с помощью функции fillna()
X_train = X_train.fillna(0)

# масштабирование признаков обучающей выборки
stdSc = StandardScaler()
X_train = stdSc.fit_transform(X_train)

In [22]:
# логистическая регрессия с L2 регуляризацией после удаления категориальных признаков
logistic_regressor_tuning(X_train, Y_train)

Time elapsed: 0:01:08.269124
Mean train scores for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]: [ 0.71225083  0.71761715  0.71797864  0.71798888  0.7179892   0.71798926
  0.71798927]
Mean test scores for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]: [ 0.71133915  0.7163758   0.71655939  0.71653424  0.71653036  0.71652997
  0.71652989]
Mean fit time for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]: [ 0.84740167  1.52660217  1.9234066   2.10720344  2.1244029   2.13600302
  2.10240312]
Optimal C and score on test data: 0.01 [ 0.71655939]


# Ответ 2:

Качество логистической регрессии после удаления категориальных признаков при С=0.01 составило 0.71655939. Отличия от качества модели над всеми признаками наблюдается начиная с 6-го знака после запятой. Можно заключить, что удаление катеориальных признаков никак модель не улучшило.

In [12]:
# количество различных идентификаторов героев в игре
heroes_data = train_data.loc[:, ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
                       'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
count_unique = np.unique(heroes_data)
print("Number of unique labels of heroes in train data:", len(count_unique))
print(count_unique)
print('='*40)

heroes_game = pd.read_csv('heroes.csv')
print ('Number of unique labels of heroes in game:', len(heroes_game))

Number of unique labels of heroes in train data: 108
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 109 110 112]
Number of unique labels of heroes in game: 112


# Ответ 3:

Всего в игре может существовать 112 игроков, в данных функция unique() нашла 108 различных меток героев.

In [23]:
# Код для формирования "мешка слов" по героям
# N — количество различных героев в выборке
def bag_words (data, X): 
    N = 112
    X_pick = np.zeros((data.shape[0], N))

    for i, match_id in enumerate(data.index):
        for p in range(5):
            X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

    # Добавим полученные числовые признаки из данных к обучающей выборке
    X = np.concatenate((X, X_pick), axis=1)
    return X

X_train = bag_words(train_data, X_train)

In [24]:
# логистическая регрессия с L2 регуляризацией после удаления категориальных признаков
logistic_regressor_tuning(X_train, Y_train)

Time elapsed: 0:02:03.464229
Mean train scores for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]: [ 0.72625999  0.74831035  0.75434344  0.75472418  0.75473198  0.75473211
  0.75473206]
Mean test scores for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]: [ 0.7250715   0.74633415  0.75173774  0.75194752  0.7519276   0.75192526
  0.75192469]
Mean fit time for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]: [ 0.87960157  1.804003    3.03880434  4.1980073   4.40981131  4.37260666
  4.4016089 ]
Optimal C and score on test data: 0.1 [ 0.75194752]


# Ответ 4:

Качество логистической регрессии улучшилось после добавления "мешка слов" и при оптимальном С=0.1 составило 0.75194752. Добавление новых признаков улучшает качество модели.

In [25]:
# предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей
test_data = pd.read_csv('features_test.csv', index_col='match_id')
drop_data = test_data.loc[:, ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', \
                       'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']] 
X_test = test_data.drop(drop_data, axis=1)
X_test = X_test.fillna(0)
X_test = stdSc.transform(X_test)
X_test = bag_words(test_data, X_test)

clf = LogisticRegression(C=0.1, random_state=42)
clf.fit(X_train, Y_train)
Y_pred = clf.predict_proba(X_test)[:, 1]
    
header = "match_id" + ',' + "radiant_win"
result = np.array([test_data.index, Y_pred])

np.savetxt('results/submission.csv', result.T, fmt='%d,%.6f', newline='\n', header=header, comments="")

# Ответ 5:

Точность на kaggle составила 0.75529